### Model Training

In [0]:
# Initiate YAML file
dataset_yaml = "/dbfs/FileStore/rachlenn/video/LabelledFishYOLOv11/dataset.yaml"

yaml_content = """
path: /dbfs/FileStore/rachlenn/video/LabelledFishYOLOv11
train: train/images
val: valid/images
nc: 1
names: ['fish']
"""

with open(dataset_yaml, "w") as f:
    f.write(yaml_content)



In [0]:
from ultralytics import YOLO

# Load pre-trained YOLOv8 nano model
model = YOLO("yolov8n.pt")


In [0]:
import os
import torch

# CPU-only
os.environ["CUDA_VISIBLE_DEVICES"] = ""  

In [0]:
results = model.train(
    data=dataset_yaml,
    epochs=50,
    imgsz=640,
    batch=16,
    project="/dbfs/FileStore/rachlenn/video/LabelledFishYOLOv11/runs/train",
    name="fish_yolo",
    exist_ok=True,
    device="cpu",   
    workers=0,
    single_cls=True
)
